In [1]:
import pandas as pd
import numpy as np
import glob
import os

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/piyush/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
df = pd.read_csv('../data/paris_reviews.csv')

In [3]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)


def remove_urls(text):
    """Remove urls from a string"""
    import re
    return re.sub(r'http\S+', '', text, flags=re.MULTILINE)


In [4]:
df['comments'] = df['comments'].astype(str)
df['comments'] = df['comments'].apply(remove_html_tags)
df['comments'] = df['comments'].apply(remove_urls)
df['date']= pd.to_datetime(df['date'])

In [5]:
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,5396,4824,2009-06-30,19995,Sarah,Perfect location!! Nasrine was a delight and m...
1,5396,4968,2009-07-03,20117,Chris,This is a nice place in a great location in Pa...
2,5396,5240,2009-07-08,22190,Annelaure,"Nice studio, very clean, very quiet, in a grea..."
3,5396,9619,2009-09-10,11947,Jean,"Superb location, great studio. \rA slice of Pa..."
4,5396,18970,2009-12-02,40625,Bette,"Perfect place to be in Paris, walking home eve..."


In [14]:
df['date'] = pd.to_datetime(df.date)
df['year'] = df['date'].dt.year

In [18]:
df.year.value_counts()

year
2022    394577
2019    220245
2018    174277
2021    161016
2017    122830
2020     83678
2016     82636
2023     77870
2015     52798
2014     23088
2013      9167
2012      3386
2011      1031
2010       240
2009         6
Name: count, dtype: int64

In [9]:
df = df[df['year'].isin([2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023])].reset_index(drop=True)

In [10]:
df.year.value_counts()

year
2022    394577
2019    220245
2018    174277
2021    161016
2017    122830
2020     83678
2016     82636
2023     77870
2015     52798
2014     23088
2013      9167
Name: count, dtype: int64

In [12]:
df = df.groupby('year').apply(lambda x: x.sample(5000)).reset_index(drop=True)

,listing_id,id,date,reviewer_id,reviewer_name,comments,year
0,1268148,7258925,2013-09-13,8319657,Annia,Thank you Genia for a wonderful stay in Paris!...,2013
1,756756,5355316,2013-06-25,6909807,Freddy,It was our first time using a place from bnb a...,2013
2,408145,8256849,2013-10-22,712966,Kanako,My BF and I stayed at Christine's flat for 5 d...,2013
3,622120,8431809,2013-10-30,8890397,Vladimir,Very good location to explore the real Paris. ...,2013
4,746611,6389893,2013-08-11,5489294,Colin,"First, the location can not be beat: quiet, up...",2013
...,...,...,...,...,...,...,...
54995,474725,794739605823497221,2023-01-01,25857797,Elijah,"Overall my family of 5 (myself, wife, and 3 yo...",2023
54996,643341295891225983,823067169930095754,2023-02-09,464630702,Laëtitia,"Excellent séjour, hôte très sympathique.",2023
54997,51415710,802000990527917011,2023-01-11,486080678,Stefano,Posizione ottima vicino la fermata della metro...,2023
54998,23959137,830975498237172235,2023-02-20,499087688,微信用户,棒极了,2023


In [ ]:
from spacy_langdetect import LanguageDetector
import spacy
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True) #2
text_content = "Er lebt mit seinen Eltern und seiner Schwester in Berlin."
doc = nlp(text_content) #3
detect_language = doc._.language #4
print(detect_language)

In [ ]:
from spacy_langdetect import LanguageDetector
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
df.listing_id.nunique()

In [ ]:
test = df.groupby('listing_id')['comments'].apply(list).reset_index(name='new')
docs = test['new'].apply(lambda x : " ".join(x)).values

In [ ]:
docs = test['new'].apply(lambda x : " ".join(x)).values

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
test = df.groupby('listing_id')['comments'].apply(list).reset_index(name='new')
docs = test['new'].apply(lambda x : " ".join(x)).values

In [ ]:
len(docs)

In [ ]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']




In [ ]:
docs = docs[0:100]
docs

In [ ]:

topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)

In [ ]:
topic_model = BERTopic().fit(docs)


In [ ]:
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
topic_model.save("model_dir", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)




In [ ]:
loaded_model = BERTopic.load("/Users/piyush/Desktop/dsml_Portfolio/new_project/model_dir")


In [ ]:
loaded_model

In [ ]:
loaded_model.get_topic_info()

In [ ]:
topics, probs = topic_model.fit_transform(docs)


In [ ]:
probs

In [ ]:
loaded_model.probabilities_